In [1]:
import pptk
import numpy as np

In [3]:
# load point cloud  
# points = np.load(r"D:\484_final_project\processed_point_cloud\slam05_mapping_global.npy")
points = np.load(r"C:\Users\kkhua\Desktop\ECE484\Final Project\slame06_mapping_global.npy")
# xyz = points[:, :, :3]
# intensity = points[:,:, 4]

In [17]:
points_0 = np.load(r"D:\484_final_project\processed_point_cloud\slam01_mapping_global.npy")

In [4]:
points.shape

(568, 128, 1024, 7)

In [60]:
predictions_005 = np.load('slam03_predictions_005.npy')

In [41]:
predictions_01 = np.load('slam03_predictions_01.npy')

In [7]:
predictions = np.load(r"C:\Users\kkhua\Desktop\ECE484\Final Project\slam06_laneline.npy")

In [71]:
# point cloud has shape (frames, channels, points, attributes)
predictions.shape

(568, 131072)

In [8]:
# create the viewer with intensity as the color
frame = 0
frame_end = 1
u = pptk.viewer(points[frame:frame_end,:,:,:3])
u.attributes(
             points[frame:frame_end,:,:,3].flatten(),
             points[frame:frame_end,:,:,4].flatten(),
             points[frame:frame_end,:,:,5].flatten(),
             points[frame:frame_end,:,:,6].flatten(),
            #  points[frame:frame_end,:,:,6].flatten()*median_scaling,
            #  predictions_005*100000
             predictions[frame:frame_end,:].flatten()*100000,
            #  predictions_005[frame:frame_end,:].flatten()*100000
            #  predictions_01[frame:frame_end,:].flatten()*100000
             )
u.set(point_size = 0.01)

In [72]:
# create the viewer with intensity as the color
frame = 0
frame_end = 568
x= pptk.viewer(points_0[frame:frame_end,:,:,:3])
x.attributes(
             points_0[frame:frame_end,:,:,3].flatten(),
             points_0[frame:frame_end,:,:,4].flatten(),
             points_0[frame:frame_end,:,:,5].flatten(),
             points_0[frame:frame_end,:,:,6].flatten(),
             points_0[frame:frame_end,:,:,6].flatten()*10,
            #  predictions_005*100000
            #  predictions[frame:frame_end,:].flatten()*100000,
            #  predictions_005[frame:frame_end,:].flatten()*100000
            #  predictions_01[frame:frame_end,:].flatten()*100000
             )
x.set(point_size = 0.01)
x.color_map('jet', scale = [0,65532])

MemoryError: 

In [70]:
u.color_map('jet', scale = [0,6552])

In [53]:
u.color_map('jet', scale = [0,6532])

In [41]:
u.set(point_size = 0.001)

In [23]:
target_intensities = points[frame:frame_end,:,:,6].flatten()

In [ ]:
reference_intensities = points_0[frame:frame_end,:,:,6].flatten()

In [35]:
np.mean(reference_intensities)

6803.8356315762385

In [28]:
# Get scaling factor using different methods
mean_scaling = get_scaling_factor(reference_intensities, target_intensities, method='mean')
median_scaling = get_scaling_factor(reference_intensities, target_intensities, method='median')

# Apply the scaling factor
normalized_intensities = target_intensities * mean_scaling

# Evaluate the results
evaluate_scaling(reference_intensities, target_intensities, mean_scaling)


Scaling factor: 5.408
Original mean - Reference: 6803.836, Target: 1258.144
Scaled mean - Reference: 6803.836, Scaled Target: 6803.836
Original max - Reference: 65535.000, Target: 65535.000
Scaled max - Reference: 65535.000, Scaled Target: 354402.483


In [59]:
mean_scaling

5.407835244411551

In [17]:
np.mean(points[frame:frame_end,:,:,6].flatten())

681.4610427856445

In [9]:
pred = np.load(r"C:\Users\Bryan\Desktop\cs411\pred.npy")

In [10]:
pred.shape

(131072,)

In [38]:
u.set(point_size = 0.001)

In [42]:
# save the selcted points (run this to save the current progress)
selected_laneline = u.get('selected')

In [43]:
# create the label array with shape of points in a single frame 128 x 1024 
lane_line_label = np.zeros_like(points[frame:frame+35, :, :, 4].flatten())

In [44]:
total_labels = len(lane_line_label)

In [46]:
total_labels//35

131072

In [54]:
# resume the selected lines on pptk viewer (optional)
u.set(selected = selected_laneline)

In [47]:
# set the label array with the selected_laneline
lane_line_label[selected_laneline] = 1
# create a label folder if you have not
# np.save(f'label/lane_line_label_{frame}', lane_line_label)

In [50]:
lane_line_label.reshape(35,131072)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
lane_line_label.shape

(4587520,)

In [52]:
split_size = 131072
for i in range(35):
    # Calculate start and end indices
    start_idx = i * split_size
    end_idx = start_idx + split_size
    
    # Extract the chunk
    chunk = lane_line_label[start_idx:end_idx]
    
    # Save to file
    np.save(f'label/label_{i:02d}.npy', chunk)

In [27]:
import numpy as np

def get_scaling_factor(reference_array, target_array, method='mean'):
    """
    Calculate a single scaling factor to normalize target array to reference array.
    
    Parameters:
    reference_array: np.array - The reference intensity values (recorded in first condition)
    target_array: np.array - The intensity values to be normalized (recorded in second condition)
    method: str - Method to calculate scaling factor ('mean', 'median', 'max', 'percentile')
    
    Returns:
    float - Scaling factor to multiply with target array
    """
    if method == 'mean':
        # Use mean ratio
        scaling_factor = np.mean(reference_array) / np.mean(target_array)
    
    elif method == 'median':
        # Use median ratio (more robust to outliers)
        scaling_factor = np.median(reference_array) / np.median(target_array)
    
    elif method == 'max':
        # Use maximum value ratio
        scaling_factor = np.max(reference_array) / np.max(target_array)
    
    elif method == 'percentile':
        # Use 95th percentile ratio (robust to extreme outliers)
        scaling_factor = (np.percentile(reference_array, 95) / 
                         np.percentile(target_array, 95))
    
    else:
        raise ValueError("Method must be one of: 'mean', 'median', 'max', 'percentile'")
        
    return scaling_factor

# Function to evaluate the scaling
def evaluate_scaling(reference_array, target_array, scaling_factor):
    """Print statistics about the scaling results"""
    scaled_array = target_array * scaling_factor
    
    print(f"Scaling factor: {scaling_factor:.3f}")
    print(f"Original mean - Reference: {np.mean(reference_array):.3f}, Target: {np.mean(target_array):.3f}")
    print(f"Scaled mean - Reference: {np.mean(reference_array):.3f}, Scaled Target: {np.mean(scaled_array):.3f}")
    print(f"Original max - Reference: {np.max(reference_array):.3f}, Target: {np.max(target_array):.3f}")
    print(f"Scaled max - Reference: {np.max(reference_array):.3f}, Scaled Target: {np.max(scaled_array):.3f}")